# Feature engineering

Авторы: Анастасия Никольская, Антон Першин, Гирдюк Дмитрий

Датасет скачивать по ссылке: https://disk.yandex.ru/d/cwL3Ka4ECyQwpw

## Импорты 

In [ ]:
import yaml

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler


with open("../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

### Общая информация

In [ ]:
df_train = pd.read_csv(cfg["house_prices"]["train_dataset"])
df_train.head()

Не все столбцы здесь выведены. Их список мы можем получить, используя аттрибут `columns`:

In [ ]:
df_train.columns

Почистим данные в нескольких столбцах, основываясь на 'data_description.txt' датасета

In [ ]:
df_train["Exterior2nd"] = df_train["Exterior2nd"].replace({"Brk Cmn": "BrkComm"})

# some values of GarageYrBlt are corrupt, so we'll replace them with the year the house was built
df_train["GarageYrBlt"] = df_train["GarageYrBlt"].where(df_train.GarageYrBlt <= 2010, df_train.YearBuilt)

# names beginning with numbers are awkward to work with
df_train.rename(
    columns={"1stFlrSF": "FirstFlrSF", "2ndFlrSF": "SecondFlrSF", "3SsnPorch": "Threeseasonporch"},
    inplace=True,
)

In [ ]:
df_num = df_train.select_dtypes(exclude=["object"])
df_cat = df_train.select_dtypes(include=["object"])

## Коррелирующие признаки

In [ ]:
fig, axes = plt.subplots(8, 5, figsize=(20, 20))
axes_flattened = axes.reshape(-1)
for i in range(len(df_num.columns)):
    ax = axes_flattened[i]
    sns.scatterplot(
        x=df_num.iloc[:, i],
        y="SalePrice",
        data=df_num.dropna(),
        ax=ax,
    )
fig.tight_layout(pad=1.0)

In [ ]:
def corr_plot(df: pd.DataFrame, method: str = "pearson", annot: bool = True, **kwargs) -> None:
    sns.clustermap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )

corr_plot(df_num, annot=None)

Из этой матрицы можно увидеть, какие столбцы сильно коррелируют между собой, например:
1. GarageYrBlt и YearBuilt
2. TotRmsAbvGrd и GrLivArea
3. FirstFlrSF и TotalBsmtSF
4. GarageArea и GarageCars

In [ ]:
df_train.drop(
    ["GarageYrBlt", "TotRmsAbvGrd", "FirstFlrSF", "GarageCars"],
    axis=1,
    inplace=True
)

## Заполнение пустых значений

Может быть множество вариантов, при которых строка может содержать пустые значения. Например:
1. Дом с 2 спальнями не может включать ответ на вопрос, насколько велика третья спальня
2. Кто-то из опрошенных может не делиться своим доходом

Библиотеки Python представляют недостающие числа как NaN-ы, что является сокращением от "not a number".

Соберем статистику, связанную с NaN-ми.

In [ ]:
df_nan = (df_train.isnull().mean() * 100).reset_index()
df_nan.columns = ["column_name", "percentage"]
df_nan.sort_values("percentage", ascending=False, inplace=True)
df_nan.head(20)

Выведем квантили:

In [ ]:
for percent in (80, 50, 20, 5):
    print(f"Number of columns with more than {percent}% NANs: {(df_nan.percentage > percent).sum()}")

Выведем столбцы с более чем 80% NaN-в

In [ ]:
nan_columns = list(df_nan[df_nan.percentage > 80]["column_name"])
nan_columns

Большинство моделей не умеют работать с NaN-ми. Поэтому требуется избавиться от них.

### Выброс столбцов с NaN-ми

In [ ]:
# 1 вариант - если, например, нужно выбросить одинаковые столбцы для обучающей и тестовой выборок
num_сols_with_missing = [
    col for col in df_num.columns if df_num[col].isnull().any()
]
num_сols_with_missing

In [ ]:
print(len(df_num.columns))
df_num_dropped = df_num.drop(num_сols_with_missing, axis=1)
print(len(df_num_dropped.columns))

In [ ]:
# 2 вариант: выбросить столбцы, напрямую используя `dropna()`
print(len(df_num.columns))
df_num_dropped = df_num.dropna(axis=1)
print(len(df_num_dropped.columns))

Если эти столбцы содержат полезную информацию (в местах, которые не были пропущены), модель теряет доступ к этой информации при удалении столбца. Кроме того, если тестовые данные имеют отсутствующие значения в тех местах, где тренировочные не имели, это приведет к ошибке.

Так что обычно это не лучшее решение. Однако оно может быть полезно, когда большинство значений в столбце отсутствуют.

### Заполнение недостающих значений каким-то значением

Это значение будет не совсем правильным в большинстве случаев, но обычно оно дает более точные модели, чем полное удаление столбца.

**Числовые признаки**

Поведение по умолчанию заполняет столбец средним значением в заполненных ячейках. Существуют и более сложные стратегии.

In [ ]:
my_imputer = SimpleImputer()

filled_cols = my_imputer.fit_transform(df_train[num_сols_with_missing])
filled_cols

Альтернативно можно заполнить столбцы средним напрямую (или нулями, или чем угодно)

In [ ]:
df_train[num_сols_with_missing].fillna(df_train[num_сols_with_missing].mean())

С точки зрения статистики такое заполнение оправдано, если между признаками нет явной зависимости. В таком случае замена пропусков средними значениями не вносит смещения. Однако, часто условие независимости нарушается. В данном примере свойства домов сильно зависят от того, в каком районе они расположены. Поэтому средние значения лучше считать по районам.

Взглянем на распределения средних значений по районам

In [ ]:
neigh_grouped = df_train.groupby("Neighborhood")
neigh_lot = (
    neigh_grouped["LotFrontage"].mean()
    .reset_index(name="LotFrontage_mean")
)
neigh_garage = (
    neigh_grouped["GarageArea"].mean()
    .reset_index(name="GarageArea_mean")
)

fig, axes = plt.subplots(1,2,figsize=(22,8))
axes[0].tick_params(axis="x", rotation=90)
sns.barplot(x="Neighborhood", y="LotFrontage_mean", data=neigh_lot, ax=axes[0])
axes[1].tick_params(axis="x", rotation=90)
sns.barplot(x="Neighborhood", y="GarageArea_mean", data=neigh_garage, ax=axes[1])

In [ ]:
df_train["LotFrontage"] = df_train.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.mean()))
df_train["GarageArea"] = df_train.groupby("Neighborhood")["GarageArea"].transform(lambda x: x.fillna(x.mean()))

Заполним все оставшиеся числовые признаки средними (ранее мы не сохраняли результат в `df_train`)

In [ ]:
df_train[num_сols_with_missing] = df_train[num_сols_with_missing].fillna(df_train[num_сols_with_missing].mean())

Те столбцы, которые содержали более 80% NANов, удалим совсем

In [ ]:
df_train.drop(nan_columns, inplace=True, axis=1)

**Категориальные (номинальные) признаки**

Понятие среднего здесь тяжело использовать, поэтому проще заполнить модой, то есть наиболее часто встречающимся значением

In [ ]:
cols = ["MasVnrType", "MSZoning", "Exterior1st", "Exterior2nd", "SaleType", "Electrical", "Functional"]
for col in cols:
    print(f"Mode of column {col} is {df_train[col].dropna().mode()[0]}")

In [ ]:
df_train[cols] = df_train.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.dropna().mode()))

**Порядковые признаки**

Мы можем их заполнить средним или часто встречающимся, но также можно использовать значение по умолчанию "NA". Это значение будет удобно ассоциировать с нулем

In [ ]:
cat = [
    "GarageType", "GarageFinish", "BsmtFinType2", "BsmtExposure", "BsmtFinType1",
    "GarageCond", "GarageQual", "BsmtCond", "BsmtQual", "FireplaceQu", "KitchenQual",
    "HeatingQC", "ExterQual", "ExterCond"
]
df_train[cat] = df_train[cat].fillna("NA")

## Удаление признаков со слабой вариативностью

**Признаки с одним типичным значением**

Некоторые признаки в основном состоят из одного значения или нулей, что не особо полезно для нас. Поэтому мы устанавливаем пороговое значение, определяемое пользователем, на уровне 95%. Если столбец имеет более 95% от одного и того же значения, мы считаем признак бесполезными и удалим его.

In [ ]:
def get_almost_constant_columns(df: pd.DataFrame, dropna: bool = True) -> list[str]:
    cols = []
    for i in df:
        counts = df[i].dropna().value_counts() if dropna else df[i].value_counts()
        most_popular_value_count = counts.iloc[0]
        if (most_popular_value_count / len(df)) * 100 > 95:
            cols.append(i)

    return cols

In [ ]:
df_cat = df_train.select_dtypes(include=["object"])
overfit_cat = get_almost_constant_columns(df_cat)
df_train = df_train.drop(overfit_cat, axis=1)
overfit_cat

In [ ]:
df_num = df_train.select_dtypes(exclude=["object"])
overfit_num = get_almost_constant_columns(df_num, dropna=True)
df_train = df_train.drop(overfit_num, axis=1)
overfit_num

**Признаки с маленькой дисперсией**

Другой способ - использовать метод VarianceThreshold от sklearn — это простой базовый подход к выбору признаков. Он удаляет все признаки, дисперсия которых не соответствует определенному порогу. По умолчанию он удаляет все элементы с нулевой дисперсией, т.е. те элементы, которые имеют одинаковое значение у всех семплов.

Стоит отметить, что дисперсия является абсолютной величиной, и выбор порога в этом случае является эмпирическим. При этом в общем случае малые значения дисперсии не говорят о бесполезности признака. Если признак задан на поле вещественных чисел, то его дискриминирующая способность не зависит от дисперсии, так как любой непрерывный интервал на вещественной оси содержит бесконечный набор значений. Однако, в случае дискретных значений (пример, целочисленных признаков) VarianceThreshold действительно становится полезным

In [ ]:
from sklearn.feature_selection import VarianceThreshold
fs = VarianceThreshold(threshold=0.1)
num_col = df_train.select_dtypes(exclude=["object"])

fs.fit(num_col)  # fit finds the features with low variance
sum(fs.get_support())

Метод `get_support()` возвратит булевскую маску для признаков, которые проходят указанный порог по дисперсии. Ее можно использовать для отбора этих признаков 

In [ ]:
fs.get_support()

Например, таким образом мы получаем список всех признаков, которые были отсеяны данным алгоритмом:

In [ ]:
num_col.columns[~fs.get_support()]

## Удаление выбросов

Удаление выбросов предотвратит воздействие экстремальных значений на производительность наших моделей.

Из скаттерплотов выше мы можем увидеть, что следующие признаки имеют экстремальные выбросы:

* LotFrontage
* LotArea
* BsmtFinSF1
* TotalBsmtSF
* GrLivArea

Мы уберем выбросы на основе определенного порогового значения.
Эти значения мы получим из боксплотов ("ящик с усиками").

In [ ]:
out_col = ["LotFrontage", "LotArea", "BsmtFinSF1", "TotalBsmtSF", "GrLivArea"]

fig, axes = plt.subplots(1, 5, figsize=(20, 5))
for ax, col in zip(axes, out_col):
    sns.boxplot(y=df_train[col], data=df_train, ax=ax)
    
fig.tight_layout(pad=1.5)

In [ ]:
for col, upper_bound in (
    ("LotFrontage", 200),
    ("LotArea", 100000),
    ("BsmtFinSF1", 4000),
    ("TotalBsmtSF", 5000),
    ("GrLivArea", 4000),
):
    df_train = df_train.drop(df_train[df_train[col] > upper_bound].index)

После удаления выбросов, сильно коррелированных признаков и условных отсутствующих значений мы можем приступить к добавлению дополнительной информации для обучения нашей модели. Это делается с помощью - Feature Engineering.

## Feature Engineering

Feature Engineering - это техника, с помощью которой мы создаем новые признаки, которые потенциально могут помочь в прогнозировании нашей целевой переменной, которая в данном случае является SalePrice. 

MSSubClass - это столбец с числовым признаком, который на самом деле можно представить как категориальный

In [ ]:
df_train["MSSubClass"].value_counts()

In [ ]:
df_train["MSSubClass"] = df_train["MSSubClass"].apply(str)

In [ ]:
ordinal_map = {"Ex": 5,"Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0}
fintype_map = {"GLQ": 6,"ALQ": 5,"BLQ": 4,"Rec": 3,"LwQ": 2,"Unf": 1, "NA": 0}
expose_map = {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0}
# fence_map = {"GdPrv": 4,"MnPrv": 3,"GdWo": 2, "MnWw": 1,"NA": 0}  -- выброшен

In [ ]:
ord_col = [
    "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC", 
    "KitchenQual", "GarageQual", "GarageCond", "FireplaceQu"
]
for col in ord_col:
    df_train[col] = df_train[col].map(ordinal_map)
    
fin_col = ["BsmtFinType1", "BsmtFinType2"]
for col in fin_col:
    df_train[col] = df_train[col].map(fintype_map)

df_train["BsmtExposure"] = df_train["BsmtExposure"].map(expose_map)

- Основываясь на текущих признаках, мы можем добавить первый дополнительный признак, который будет называться TotalLot и который суммирует LotFrontage и LotArea для определения общей площади земли, доступной в виде лота.

  TotalLot = LotFrontage + LotArea

- Мы также можем рассчитать общее количество площади поверхности дома, TotalSF, сложив площадь от 1-го этажа и 2-го этажа.
  
  TotalSF = TotalBsmtSF + 2ndFlrSF
  
- TotalBath также может быть использован, чтобы сказать нам в общей сложности, сколько ванных комнат есть в доме.

  TotalBath = FullBath + HalfBath
  
- Мы также можем добавить все различные типы крылец вокруг дома и обобщить в общей площади крыльца, TotalPorch.

  TotalPorch = OpenPorchSF + EnclosedPorch + ScreenPorch

- TotalBsmtFin = BsmtFinSF1 + BsmtFinSF2

In [ ]:
df_train["TotalLot"] = df_train["LotFrontage"] + df_train["LotArea"]
df_train["TotalBsmtFin"] = df_train["BsmtFinSF1"] + df_train["BsmtFinSF2"]
df_train["TotalSF"] = df_train["TotalBsmtSF"] + df_train["SecondFlrSF"]
df_train["TotalBath"] = df_train["FullBath"] + df_train["HalfBath"]
df_train["TotalPorch"] = df_train["OpenPorchSF"] + df_train["EnclosedPorch"] + df_train["ScreenPorch"]

In [ ]:
df_train.columns

In [ ]:
df_train["LivLotRatio"] = df_train["GrLivArea"] / df_train["LotArea"]

Мы также включаем создание бинарных столбцов для некоторых признаков, которые могут указывать на наличие(1) / отсутствие(0) некоторых признаков дома

In [ ]:
cols = [
    "MasVnrArea", "TotalBsmtFin", "TotalBsmtSF", "SecondFlrSF", "WoodDeckSF", "TotalPorch"
]
for col in cols:
    col_name = col + "_bin"
    df_train[col_name] = df_train[col].apply(lambda df_train: 1 if df_train > 0 else 0)

В нашем наборе достаточно много категориальных признаков, и использовать их в таком виде, как они представлены в датасете, скорее всего нельзя. Это связано с тем, что модели в большинстве своем работают с евклидовыми или метрическими пространствами. Для перевода категориальных признаков в них используются различные техники, рассмотрим некоторые из них

**Label Encoding**

Естественным решением такой проблемы было бы однозначное отображение каждого значения в уникальное число. К примеру, мы могли бы преобразовать признак Street так: Pave в 0, а Grvl в 1. Эту простую операцию приходится делать часто, поэтому в модуле sklearn.preprocessing  именно для этой задачи реализован класс LabelEncoder. 

Метод fit этого класса находит все уникальные значения и строит таблицу для соответствия каждой категории некоторому числу, а метод transform непосредственно преобразует значения в числа. После fit у label_encoder будет доступно поле classes_, содержащее все уникальные значения.

In [ ]:
df_train["LandContour"].value_counts().plot.barh()

In [ ]:
label_encoder = LabelEncoder()

encoded_neigh = pd.Series(label_encoder.fit_transform(df_train["LandContour"]))
sns.histplot(encoded_neigh)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(22,8))
axes[0].tick_params(axis="x", rotation=90)
sns.histplot(df_train["LandContour"], ax=axes[0])
axes[1].tick_params(axis="x", rotation=90)
sns.histplot(encoded_neigh, ax=axes[1])

Что произойдет, если у нас появятся данные с другими категориями? LabelEncoder выдаст ошибку, что в словаре нет такой категории

In [ ]:
label_encoder.transform(df_train["LandContour"].replace("Low", "low"))

Таким образом, при использовании этого метода нужно быть уверенным, что признак не может принимать неизвестных ранее значений. 

Основная проблема такого представления заключается даже не в этом, а в том, что числовой код создал евклидово представление для данных. Это значит, что теперь можно вычесть "Low" из "Bnk" и т.д. Поэтому, например, методы, основанные на расстоянии, становятся больше неприменимы.

**One Hot encoding**

One Hot encoding является наиболее распространенным подходом для преобразования категориальных признаков, и он работает очень хорошо, если ваша категориальная переменная принимает небольшое количество значений (т.е. вы, как правило, не будете этого делать для переменных, которые принимают более 15 различных значений)

Предположим, что некоторый признак может принимать 10 разных значений. В этом случае One Hot Encoding подразумевает создание 10 признаков, все из которых равны нулю за исключением одного. На позицию, соответствующую численному значению признака мы помещаем 1.
Этот метод реализован в sklearn.preprocessing в классе OneHotEncoder. По умолчанию OneHotEncoder преобразует данные в разреженную матрицу, чтобы не расходовать память на хранение многочисленных нулей. Однако в нашем случае размер данных не является проблемой, поэтому мы будем использовать "плотное" представление.


In [ ]:
onehot_encoder = OneHotEncoder(sparse_output=False)

encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(df_cat))
encoded_categorical_columns.head()

In [ ]:
onehot_encoder.categories_

Кроме того, можно сразу удалить категории, которые встречаются редко. Это можно сделать, задав значение параметра min_frequency

In [ ]:
onehot_encoder = OneHotEncoder(sparse_output=False, min_frequency=0.3)
encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(df_cat))
encoded_categorical_columns.head()

Для категориальных столбцов в pandas можно применить one-hot-encoding с помощью метода get_dummies().

In [ ]:
len(df_train.columns)

In [ ]:
df_train = pd.get_dummies(df_train)

In [ ]:
len(df_train.columns)

One-hot-encoding-ом и label encoding-ом выбор не ограничен. Существует достаточно много альтернативных вариантов преобразования категориальных переменных. Если тема заинтересовала, обратите внимание на библиотеку [category-encoders](https://pypi.org/project/category-encoders/).

## Нормализация/Скейлинг
RobustScaler - это метод преобразования, который удаляет медиану и масштабирует данные в соответствии с диапазоном квантиля (по умолчанию IQR: межквартильный диапазон). IQR - это диапазон между 1-м квартилем (25-й квантилем) и 3 Квартиль (75-й квантиль). Он также устойчив к выпадающим значениям, что делает его идеальным для данных, где слишком много выпадающих значений, что резко сократит количество обучающих данных.

Запуская скейлер как на тренировочном, так и на тестовом наборах, мы подвергаем себя проблеме утечки данных. Утечка данных - это проблема, когда для создания модели используется информация извне набора для обучения. Если мы подгоняем скейлер как на тренировочные, так и на тестовые данные, наши характеристики тренировочных данных будут содержать распределение нашего тестового набора. Таким образом, мы неявно передаем информацию о наших тестовых данных в окончательные тренировочные данные для обучения, что не даст нам возможности по-настоящему протестировать нашу модель на данных, которые она никогда не видела.

In [ ]:
df_train.columns

In [ ]:
cols = df_train.select_dtypes(np.number).columns
# df_train = df_train.drop(["Id"], axis=1)
transformer = RobustScaler().fit(df_train[cols])
df_train[cols] = transformer.transform(df_train[cols])

Таким образом, мы сильно преобразовали наш обучающий набор. кроме перчисленного, полезно использовать PCA, выбор признаков на основе информации и други методы. Как вы, наверное, заметили, все преобразования были сделаны только для тренировочного набора, но то же самое необходимо сделать и для тестового.

Чтобы предотвратить утечку данных, все преобразования по среднему и т.п. нужно сделать независимо, а если мы, например, кодировали или удаляли столбцы, нужно сделать такое же преобразование, используя старые правила. 

In [ ]:
df_train.head()